In [1]:
import sys
sys.path.append('../')

from src.layers.linear import Linear
from src.activation.functional import ReLU, Softmax, MSELoss
from src.optim.optim import SGD
from src.utils.model import ModelClass

import numpy as np

In [2]:
class MLP(ModelClass):
    def __init__(self, input_size, output_size, hidden_size):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.layers = [Linear(input_size, hidden_size),
                       ReLU(), 
                       Linear(hidden_size, output_size),
                       Softmax()
                       ]
        
    def __call__(self, x: np.ndarray) -> np.ndarray:
        assert len(self.layers) > 0
        for layer in self.layers:
            x = layer(x)
        return x

In [3]:
model = MLP(input_size=5, output_size=10, hidden_size=10)

In [4]:
inputs = np.array([[0.5, 0.7, 0.3, 0.1, 0.8],
                   [0.1, 0.6, 0.9, 0.01, 0.4]])
gt = np.array([2])

In [5]:
model(inputs)

array([[0.11030189, 0.08927539, 0.1190022 , 0.11251624, 0.07165914,
        0.09748807, 0.08633985, 0.15929211, 0.08193449, 0.07219062],
       [0.10972655, 0.08591191, 0.12106468, 0.11440065, 0.08015815,
        0.09476197, 0.08071493, 0.16425397, 0.07861627, 0.07039092]])